In [1]:
import numpy as np

# Orientations

We are interested in representing the orientation of crystals in 3D.

## Rotation and orientation matrix

A rotation matrix, when multiplied by a vector, results in the same vector rotated. An orientation matrix, when multiplied by a vector, results in the same vector in the rotated coordinate system. The orientation matrix (*passive rotation*) is the transpose of the corresponding rotation matrix (*active rotation*).

It is trivial to define rotation or orientation matrices which express rotations in a plane. For example, we will calculate the rotation matrices for 60 degrees around the Z and X axes. First, we need the angle in radians:

In [2]:
def rad(a):
    # a is the angle in degrees, returns angle in radians
    return (a*np.pi)/180
ang=rad(60) # rotation angle: 60 degrees
print(ang)

1.0471975511965976


To find the rotation matrix, we take into account that the vector component parallel to the rotation axis must remain unchanged. The other components get multiplied by the sine or the cosine. Examples:

In [3]:
# Rotation around axis Z (in the XY plane) with angle a in radians
def rot_z(a):
    c,s=np.cos(a),np.sin(a)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]])
rz=rot_z(ang)
print(rz)

[[ 0.5       -0.8660254  0.       ]
 [ 0.8660254  0.5        0.       ]
 [ 0.         0.         1.       ]]


In [4]:
# Rotation around axis X (in the YZ plane) with angle a in radians
def rot_x(a):
    c,s=np.cos(a),np.sin(a)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]])
rx=rot_x(ang)
print(rx)

[[ 1.         0.         0.       ]
 [ 0.         0.5       -0.8660254]
 [ 0.         0.8660254  0.5      ]]


Transformation (or orientation) matrices are obtained transposing the rotation matrices.

In [18]:
tz=np.transpose(rz)
tx=np.transpose(rx)
print(tz)
print(tx)

[[ 0.5        0.8660254  0.       ]
 [-0.8660254  0.5        0.       ]
 [ 0.         0.         1.       ]]
[[ 1.         0.         0.       ]
 [ 0.         0.5        0.8660254]
 [ 0.        -0.8660254  0.5      ]]


It is useful to check that the properties of rotation matrices are fulfilled:

In [21]:
def test_Rzx():
    rz,rx=rot_z(np.pi/2),rot_x(np.pi/2)
    assert((1,1)==(np.linalg.det(rz),np.linalg.det(rx)))              # matrix determinant == 1
    assert(np.isclose(np.identity(3),rz@np.transpose(rz)).all())      # matrix transpose == matrix inverse
    assert(np.isclose(np.identity(3),rx@np.transpose(rx)).all())
    assert(np.isclose(np.array([[0,-1,0],[1,0,0],[0,0,1]]),rz).all()) # matrix columns == rotated base vectors
    assert(np.isclose(np.array([[1,0,0],[0,0,-1],[0,1,0]]),rx).all())
test_Rzx()

## Euler angles

A triplet of Euler angles $(\varphi_1,\Phi,\varphi_2)$ represents an orientation corresponding to three successive rotations with respect to a reference position. The first rotation is performed around the Z axis with an angle $\varphi_1$. This results in the rotation of the axes X and Y, which will become X' and Y'. Next, a rotation with an angle $\Phi$ is performed around the axis X', resulting in the rotation of the axes Y' and Z, which become Y'' and Z'. Finally, a rotation with angle $\varphi_2$ is performed around the axis Z'.

With three Euler angles, we can describe any orientation in 3D. However, an orientation can be represented by more than one set of Euler angles. As a trivial example, any two triplets of Euler angles for which $\Phi$ is $0$ and $\varphi_1+\varphi_2$ is constant will represent the same orientation.

### Orientation matrix from Euler angles

The functions `rot_z` and `rot_x` previously defined can be used to define rotations around the Z and X axes. Moreover, rotations can be combined simply multiplying the corresponding matrices. Finally, we can convert a rotation matrix into an orientation matrix just transposing it. If we put everything together in a function, this function will return the orientation matrix corresponding to a triplet of Euler angles:

In [7]:
def rot(ea):
    # ea are the Euler angles as a numpy array of shape (3)
    # return orientation matrix (numpy array of shape (3,3))
    phi1,Phi,phi2=ea
    r=rot_z(phi2)@rot_x(Phi)@rot_z(phi1)
    return np.transpose(r)
ea=[rad(a) for a in [30,15,45]]
r=rot(ea)
print(r)

[[ 0.27086608  0.95387879  0.12940952]
 [-0.94505974  0.23795296  0.22414387]
 [ 0.1830127  -0.1830127   0.96592583]]


## Axis and angle

To calculate the angle of rotation we take into account that the trace of the rotation (or orientation) matrix, for an angle of rotation $\theta$, must be equal to $1+2\cos(\theta)$. Therefore, to get the angle in degrees from a rotation matrix:

In [8]:
def ang(r):
    # r is the orientation matrix (numpy array shape (3,3))
    # returns angle in radians
    return np.arccos((np.trace(r)-1)/2.)
def deg(a):
    # a is the angle in radians, returns angle in degrees
    return (a*180)/np.pi
print(deg(ang(r)))

76.26848888646785


[To find the axis](https://en.wikipedia.org/wiki/Rotation_matrix#Determining_the_axis), we take into account that rotating the vector that represents the axis must result in the same vector. This is equivalent to say that the skew-symmetric part of the rotation matrix (obtained as $1/2(R-R^T)$) corresponds to the components of the axis. We then use `norm` to normalize the obtained normal vector.

In [9]:
def norm(v):
    # v is a vector (numpy array of 1D)
    # returns normalized vector (such that \v\ is 1)
    return v/(v*v).sum()**0.5
def axis(r):
    # r is an orientation matrix (numpy array of shape (3,3))
    # returns the axis as numpy array of sahpe (3)
    a=0.5*(r-np.transpose(r))
    return norm(np.array([r[1,2],r[2,0],r[0,1]]))
print(axis(r))

[0.22486246 0.18359943 0.95693685]


**TODO** Calculate [rotation matrix from axis-angle](https://en.wikipedia.org/wiki/Rotation_matrix#Rotation_matrix_from_axis_and_angle)

## Miller indices

To represent orientations using Miller indices we need a plane (its normal direction) and a direction in that plane.

The orientation matrix and the corresponding Miller indices are easily correlated. The leftmost column is the normal to the plane, the rightmost column the direction. Both are perpendicular, and the third unitary vector must be perpendicular to both, so it is obtained using the vector cross product (see: [Texture Components and Euler Angles](http://pajarito.materials.cmu.edu/lectures/Components_EulerAngles-14Jan20.pdf)). We also normalize the vectors so that they are unitary. They should already be, but this way we mitigate any rounding error.

In [10]:
def miller(r):
    # r is an orientation matrix (numpy array of shape (3,3))
    # returns the Miller indices for plane normal and direction as two numpy arrays
    return norm(r[:,2]),norm(r[:,0])
hkl,uvw=miller(r)
print(hkl,uvw)

[0.12940952 0.22414387 0.96592583] [ 0.27086608 -0.94505974  0.1830127 ]


Using this relationship, we can also find the rotation matrix from the Miller indices. We only need to normalize the normal and direction vectors and calculate their dot product. Additionally, we check that the direction must be on the plane (must be perpendicular to the plane normal).

In [23]:
def Rmiller(hkl,uvw):
    # hkl is a numpy array of shape 3 with components of normal to plane
    # uvw is a numpy array of shape 3 with components of direction
    # returns orientation matrix as numpy array (3,3)
    assert(0==np.dot(hkl,uvw)) # must be perpendicular
    n,d=norm(np.array(hkl)),norm(np.array(uvw))
    c=norm(np.cross(n,d))
    return np.array([d,c,n])
m=Rmiller([1,1,0],[0,0,1])
print(m)

[[ 0.          0.          1.        ]
 [ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]]


# Symmetry

Crystals usually present some form of symmetry. What this means is that they are invariant to certain rotations. For example, a cubic crystal structure (or any cube, for that matter) will present 24 equivalent orientations.

#### Example

If we can visualize three faces of a Rubik cube, we will see three different colors. Performing 90 degrees rotations around the faces of the cube, we can change these three colors. There will be a total of 24 different combinations of colors, which will correspond to the 24 equivalent orientations of cubic symmetry.

## Symmetry operators

### Cubic symmetry

There are different ways in which we can represent the symmetry operators corresponding to cubic symmetry. Eventually, what we want are 24 orientation matrices.

For example, using Miller indices, we can define the operators as the orientations corresponding to orienting each of the 6 faces of the cube parallel to the 001 direction and then considering, for each one, four perpendicular directions.

In [12]:
cubic=[
    # face 001 perpendicular to 001
    Rmiller([0,0,1],[1,0,0]),Rmiller([0,0,1],[0,1,0]),Rmiller([0,0,1],[-1,0,0]),Rmiller([0,0,1],[0,-1,0]),
    # face 010 perpendicular to 001
    Rmiller([0,1,0],[1,0,0]),Rmiller([0,1,0],[0,0,1]),Rmiller([0,1,0],[-1,0,0]),Rmiller([0,1,0],[0,0,-1]),
    # face 100 perpendicular to 001
    Rmiller([1,0,0],[0,0,1]),Rmiller([1,0,0],[0,1,0]),Rmiller([1,0,0],[0,0,-1]),Rmiller([1,0,0],[0,-1,0]),
    # face -001 perpendicular to 001
    Rmiller([0,0,-1],[1,0,0]),Rmiller([0,0,-1],[0,1,0]),Rmiller([0,0,-1],[-1,0,0]),Rmiller([0,0,-1],[0,-1,0]),
    # face -010 perpendicular to 001
    Rmiller([0,-1,0],[1,0,0]),Rmiller([0,-1,0],[0,0,1]),Rmiller([0,-1,0],[-1,0,0]),Rmiller([0,-1,0],[0,0,-1]),
    # face -100 perpendicular to 001
    Rmiller([-1,0,0],[0,0,1]),Rmiller([-1,0,0],[0,1,0]),Rmiller([-1,0,0],[0,0,-1]),Rmiller([-1,0,0],[0,-1,0])
]
for c in cubic: print(c)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
[[ 0. -1.  0.]
 [ 1.  0. -0.]
 [ 0.  0.  1.]]
[[ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  1.  0.]]
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[-1.  0.  0.]
 [ 0. -0.  1.]
 [ 0.  1.  0.]]
[[ 0.  0. -1.]
 [-1.  0.  0.]
 [ 0.  1.  0.]]
[[ 0.  0.  1.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[ 0.  0. -1.]
 [-0.  1.  0.]
 [ 1.  0.  0.]]
[[ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  0.  0.]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[ 0.  1.  0.]
 [ 1. -0.  0.]
 [ 0.  0. -1.]]
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
[[ 0. -1.  0.]
 [-1. -0. -0.]
 [ 0.  0. -1.]]
[[ 1.  0.  0.]
 [-0.  0.  1.]
 [ 0. -1.  0.]]
[[ 0.  0.  1.]
 [-1.  0.  0.]
 [ 0. -1.  0.]]
[[-1.  0.  0.]
 [-0. -0. -1.]
 [ 0. -1.  0.]]
[[ 0.  0. -1.]
 [ 1.  0.  0.]
 [ 0. -1.  0.]]
[[ 0.  0.  1.]
 [ 0.  1. -0.]
 [-1.  0.  0.]]
[[ 0.  1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]]
[[ 0.  0. -1.]


**TODO** Calculate cubic symmetry operators using Euler angles and axis-angle.

### Tetragonal symmetry

### Hexagonal symmetry

### Orthorhombic symmetry

**TODO**

## Symmetric variants

Given an orientation (represented, for example, by an orientation matrix), we can obtain all the symmetric variants corresponding to that orientation applying each of the symmetry operators. These variants represent all the different orientations that correspond to the same orientation under symmetry.

For example, we can obtain the 24 variants of `r` multiplying it by each of the 24 cubic symmetry operators in `cubic`:

In [13]:
v=[c@r for c in cubic]
for vi in v: print(vi)

[[ 0.27086608  0.95387879  0.12940952]
 [-0.94505974  0.23795296  0.22414387]
 [ 0.1830127  -0.1830127   0.96592583]]
[[-0.94505974  0.23795296  0.22414387]
 [-0.27086608 -0.95387879 -0.12940952]
 [ 0.1830127  -0.1830127   0.96592583]]
[[-0.27086608 -0.95387879 -0.12940952]
 [ 0.94505974 -0.23795296 -0.22414387]
 [ 0.1830127  -0.1830127   0.96592583]]
[[ 0.94505974 -0.23795296 -0.22414387]
 [ 0.27086608  0.95387879  0.12940952]
 [ 0.1830127  -0.1830127   0.96592583]]
[[ 0.27086608  0.95387879  0.12940952]
 [-0.1830127   0.1830127  -0.96592583]
 [-0.94505974  0.23795296  0.22414387]]
[[ 0.1830127  -0.1830127   0.96592583]
 [ 0.27086608  0.95387879  0.12940952]
 [-0.94505974  0.23795296  0.22414387]]
[[-0.27086608 -0.95387879 -0.12940952]
 [ 0.1830127  -0.1830127   0.96592583]
 [-0.94505974  0.23795296  0.22414387]]
[[-0.1830127   0.1830127  -0.96592583]
 [-0.27086608 -0.95387879 -0.12940952]
 [-0.94505974  0.23795296  0.22414387]]
[[ 0.1830127  -0.1830127   0.96592583]
 [ 0.94505974 -0.

# Misorientations

The misorientation between two orientations is the rotation that converts one orientation into the other. If these orientations are subjected to some kind of symmetry, all the symmetric variants must be considered, and the one for the lowest misorientation angle (also called *disorientation*) will be the misorientation under symmetry.

## Relative rotation

If symmetry is not considered, and we want to calculate the misorientation between the orientations $R_1$ and $R_2$, the relative rotation $\Delta R$ will transform $R_1$ into $R_2$, so $R_2=\Delta R\,R_1$ and, therefore, $\Delta R=R_2\,R_1^{-1}=R_2\,R_1^T$.

In [14]:
def dRot(r,s):
    # r and s are rotation matrices
    # returns the relative rotation
    return r@np.transpose(s)
dr=dRot(r,m)
print(dr)

[[ 0.12940952 -0.48296291  0.8660254 ]
 [ 0.22414387 -0.8365163  -0.5       ]
 [ 0.96592583  0.25881905  0.        ]]


## Axis and angle

The axis and angle of rotation can be obtained from the matrix using the functions previously defined.

In [15]:
print(deg(ang(dr)),axis(dr))

148.600285190081 [-0.42014346  0.81165484 -0.40582742]


## Crystal symmetry

Under crystal symmetry, we will need to check all the possible variants. Alternatively, we can directly apply the symmetry operators to the relative matrix. We will choose as misorientation the one for which the disorientation (or misorientation angle) is minimum.

In [16]:
def misorientation(csym,r,s):
    # csym: crystal symmetry operators (list of numpy arrys (3,3))
    # r and s are rotation matrices
    # return the angle (in radians) and axis of misorientation
    dr=dRot(r,s)
    a=2*np.pi
    for sym in csym:
        ri=sym@dr
        ai,axi=ang(ri),axis(ri)
        if ai<a: a,ax=ai,axi
    return a,ax
a,ax=misorientation(cubic,r,m)
print(deg(a),list(ax))

33.46386313025447 [0.8655052322407867, 0.22400923773979584, 0.44801847547959184]


The orientations `r` and `m` are related, under cubic symmetry, by a disorientation of 33.46 degrees around the axis (0.87,0.22,0.45).